In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

from PIL import Image
import cv2

!ls

%cd gdrive/My Drive/Graduation_Project_YOLO_tracker_trajectories_extracter

In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git  # clone recursively
%cd yolov8_tracking
!pip install -r requirements.txt  # install dependencies

In [ ]:
# The classes (and their indecies) we are going to track: car: 2, motorcycle: 3, bus: 5, truck: 7
# Using the (--save-txt) flag to save the trajectories
# Using the (--save-vid) flag to save the output video
# To get the size of the input video (So you can make sense of the output track.text file) see: https://www.linuxshelltips.com/find-video-resolution-linux/
# To do mapping from pixels to meters, take a screenshot from the input video, resize (to the same size of the video) it in krita, choose a reference object, then do the mapping
# Always resize (using the --imgsz flag) to the size of the video (just my own observation)


!python track.py --yolo-weights yolov8x.pt --tracking-method strongsort --source vlc-record-2023-02-15-19h39m38s-VID_20230215_134159.mp4-.mp4 --classes 2 3 5 7 --imgsz 1280 --save-vid --save-txt

In [ ]:
# Compress the output video

!ffmpeg -i /content/gdrive/MyDrive/Graduation_Project_YOLO_tracker_trajectories_extracter/yolov8_tracking/runs/track/exp82/vlc-record-2023-02-15-19h39m38s-VID_20230215_134159.mp4-.mp4 -vf fps=30 -vcodec libx264 /content/gdrive/MyDrive/Graduation_Project_YOLO_tracker_trajectories_extracter/compressed_output_videos/comp_output82.mp4

# **Processing the track txt file:**

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import math

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Graduation_Project_YOLO_tracker_trajectories_extracter/yolov8_tracking/runs/track/exp80/tracks/vlc-record-2023-02-15-19h39m38s-VID_20230215_134159.mp4-.txt', header = None, sep = " ", index_col = False)

In [ ]:
df.drop(df.columns[[6, 7, 8, 9, 10]], axis=1, inplace=True) # Removing the unnecessary columns

df.columns = ['frame', 'car', 'x', 'y', 'bbox_w', 'bbox_h']

# hole_cords = [620, 300]
involve_trajs_till_x_cord_ = 400

starting_x_cord = 1140 # the starting x coordinates, before which the bbox is trimmed

# To envolve the trajectories between the starting point and the hole:
df = df[df['x'] > involve_trajs_till_x_cord_]
df = df[df['x'] < starting_x_cord]

display(df)

In [ ]:
# to calculate the reduction in pixels-length, use the formula: (dist2/dist1) = (L1/L2) * factor,
# first choose a moving reference object,
# then for this object, manually measure dist2 and dist1 (from an origin point parallel to the direction of moving) to
# calculate the reduction factor ("factor"),
# then to calculate the px_to_m mapping at a given location, calculate the pixels-length of the reference object at this location,
# divide this pixels-length by the real length of the reference object.

#def get_px_len_given_target_x(ref_cord, ref_px_len, target_x):
 # target_px_len = (target_x / ref_cord[0]) * ref_px_len
  #return target_px_len

car54 = df[df['car'] == 54] # the reference car (the white tucson)
car258 = df[df['car'] == 258]
car1 = df[df['car'] == 1]
car98 = df[df['car'] == 98]
car86 = df[df['car'] == 86]


ref_x_cords = car54['x']
ref_bbox_w = car54['bbox_w']

car54

In [ ]:
def ref_px_len(x, carriageway):
  if carriageway == 'far':
    ref_px_len = ( (x - 555) * (115 - 40) / (1280 - 555) ) + 40
  else:
    ref_px_len = ( (x - 1280) * (85 - 142) / (830 - 1280) ) + 142

  return ref_px_len

ref_px_len(1003, 'close')

In [ ]:
def give_me_trajs(car_index, cars_dataset, carriageway, delta_t, fps):
  car = cars_dataset[cars_dataset['car'] == car_index]
  time_steps = []
  distance_from_last_cord = []
  speed = []
  acceleration = []
  x_cord = []
  last_cord = [car.iloc[0][2], car.iloc[0][3]]

  if carriageway == 'far':
    ref_real_len = 4.47 # the real length of the reference car of the 'far' carriageway (The white tucson)
  else:
    ref_real_len = 4.84

  count = 0

  for i in car['frame']:
    if i % (delta_t * fps) == 0:
      time_steps.append(i / fps)
      present_cord = [car.iloc[count][2], car.iloc[count][3]]
      x_cord.append(present_cord[0])
      refpxlen = ref_px_len( present_cord[0], carriageway )
      px_to_m = ref_real_len / refpxlen
      if carriageway == 'far':
        b = 10
      else:
        b = 15 # not sure about this
      delta_dist = ( math.sqrt( (present_cord[0] - last_cord[0])**2 + (present_cord[1] - last_cord[1])**2 ) - b) * px_to_m
      if len(distance_from_last_cord) == 0:
        if carriageway == 'far':
          a = 5
        else:
          a = 10 # not sure about this
        delta_dist = ( math.sqrt( (present_cord[0] - last_cord[0])**2 + (present_cord[1] - last_cord[1])**2 ) - a) * px_to_m
        if delta_dist < 0: # don't use negative distances (negativity happens due to subtracting a small number from a distance)
          delta_dist = ( math.sqrt( (present_cord[0] - last_cord[0])**2 + (present_cord[1] - last_cord[1])**2 )) * px_to_m
          print(delta_dist)
        distance_from_last_cord.append(delta_dist)
        sp = delta_dist / ( (i - car.iloc[0][0]) / fps)
        speed.append(sp) # m/sec
        acceleration.append( np.nan )
      else:
        distance_from_last_cord.append(delta_dist + distance_from_last_cord[-1])
        sp = delta_dist / delta_t
        acceleration.append( ( (sp - speed[-1]) ) / delta_t ) # m/sec^2
        speed.append(sp) # m/sec
      last_cord = present_cord
    count += 1

  car_trajs = pd.DataFrame(
      {'time(sec)': time_steps,
       'x': x_cord,
       'distance(m)': distance_from_last_cord,
       'speed(m/sec)': speed,
       'acceleration(m/sec^2)': acceleration
      })

  return car_trajs

In [ ]:
# list_of_wanted_cars = [54, 98, 149, 157, 212, 237, 243, 252]

give_me_trajs(157, df, 'far', 0.5, 30)

In [ ]:
# save the dataframe as a csv file - in the drive

car_trajs_54.to_csv('/content/gdrive/MyDrive/Graduation_Project_YOLO_tracker_trajectories_extracter/csv_trajectories_files/exp80_car54.csv')

In [ ]:
def new_distances_to_new_speeds(car_trajs, lower_limit, upper_limit, interval):
    # this function assumes that the largest distance in new_distances is smaller than the largest distance in old_distances
    # It also assumes that there's at most one new_distance less than the smallest old_distance

    old_distances = car_trajs['distance(m)']
    old_speeds = car_trajs['speed(m/sec)']

    new_distances = list(range(lower_limit, upper_limit + 1, interval))
    new_speeds = []
    skip = 1

    if(lower_limit < old_distances[0]):
        sp = old_speeds[0] - ( (old_distances[0] - lower_limit) * (old_speeds[1] - old_speeds[0]) / (old_distances[1] - old_distances[0]) )
        new_speeds.append(sp)

    else:
        skip = 0

    for i in range(len(new_distances)):
        if skip:
            skip = 0
            continue

        for j in range(len(old_distances)):
            if old_distances[j] < new_distances[i] and old_distances[j + 1] > new_distances[i]:
                sp = ( (old_speeds[j + 1] - old_speeds[j]) * (new_distances[i] - old_distances[j]) / (old_distances[j + 1] - old_distances[j]) ) + old_speeds[j]
                new_speeds.append(sp)

    return new_speeds

In [ ]:
def add_two_lists(list1, list2):
  sum_list = []
  for i in range(len(list1)):
    sum_list.append(list1[i] + list2[i])

  return sum_list

In [ ]:
def give_me_many_speeds_with_chosen_distance(df_of_cars, list_of_wanted_cars, lower_limit, upper_limit, interval, carriageway, delta_t, fps):
    output = pd.DataFrame()
    output['new_distances'] = list(range(lower_limit, upper_limit + 1, interval))
    sum_list = [0] * len(output['new_distances'])

    for car in list_of_wanted_cars:
        car_trajs = give_me_trajs(car, df_of_cars, carriageway, delta_t, fps)
        new_speeds = new_distances_to_new_speeds(car_trajs, lower_limit, upper_limit, interval)
        output[str(car)] = new_speeds
        sum_list = add_two_lists(sum_list, new_speeds)

    n = len(list_of_wanted_cars)

    avg_list = [x / n for x in sum_list]

    output['avg'] = avg_list

    return output

In [ ]:
list_of_wanted_cars = [54, 98, 149, 157, 212, 237]

output = give_me_many_speeds_with_chosen_distance(df, list_of_wanted_cars, 0, 42, 3, 'far', 0.5, 30)

output

In [ ]:
output.to_csv('/content/gdrive/MyDrive/Graduation_Project_YOLO_tracker_trajectories_extracter/csv_trajectories_files/exp80_a_lot_of_cars_3.csv')